# Assignment 3 (CS661)
## Manish Agrawal 231110028
## Nitish Kumar 231110033
### Dept(M.Tech CSE)

In [1]:
## Approach 1

import vtk
from vtk.util.numpy_support import vtk_to_numpy
import numpy as np
import matplotlib.pyplot as plt

# Function to perform Runge-Kutta 4th order (RK4) integration
def rk4_integration(vectors_func, point, step_size, max_steps, bounds):
    streamline = [point]
    for _ in range(max_steps):
        

        k1 = step_size * vectors_func(point)
        k2 = step_size * vectors_func(point + k1 / 2)
        k3 = step_size * vectors_func(point + k2 / 2)
        k4 = step_size * vectors_func(point + k3)
        next = point + (k1 + 2*k2 + 2*k3 + k4) / 6
        if not (bounds[0] <= next[0] <= bounds[1] and bounds[2] <= next[1] <= bounds[3] and bounds[4] <= next[2] <= bounds[5]):
            break  # Stop if the point is out of bounds
        streamline.append(next)
        point = next
    return streamline



def get_vector_at_point(data, point):
    probe = vtk.vtkProbeFilter()
    probe.SetSourceData(data)

    points = vtk.vtkPoints()
    points.InsertNextPoint(point.tolist())
    polydata = vtk.vtkPolyData()
    polydata.SetPoints(points)

    probe.SetInputData(polydata)
    probe.Update()

    probed_data = probe.GetOutput()
    vectors = vtk_to_numpy(probed_data.GetPointData().GetArray("vectors"))

    if vectors is None:
        return np.array([0.0, 0.0, 0.0])  # No vectors found, return zero vector
    else:
        return vectors[0]
        # Return the vector at the queried point

# Load the vector field dataset
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName("tornado3d_vector.vti")  # Use the uploaded file path
reader.Update()
data = reader.GetOutput()
bounds = data.GetBounds()


# Ask the user to input the seed point
seed_x = float(input("Enter the x-coordinate of the seed point: "))
seed_y = float(input("Enter the y-coordinate of the seed point: "))
seed_z = float(input("Enter the z-coordinate of the seed point: "))

# Create the seed point array
seed_point = np.array([seed_x, seed_y, seed_z])

# Seed point, step size, and max steps as per the assignment specifications
# seed_point = np.array([0, 0, 7])
step_size = 0.05
max_steps = 1000

# Generate streamline
streamline_forward = rk4_integration(lambda p: get_vector_at_point(data, p), seed_point, step_size, max_steps, bounds)
streamline_backward = rk4_integration(lambda p: get_vector_at_point(data, p), seed_point, -step_size, max_steps, bounds)

# Combine forward and backward streamlines
# streamline = streamline_backward[::-1][:-1] + streamline_forward
streamline = np.concatenate((streamline_backward[::-1], streamline_forward))



# Convert streamline to VTKPolyData
points = vtk.vtkPoints()
lines = vtk.vtkCellArray()

for i, point in enumerate(streamline):
    points.InsertNextPoint(point.tolist())
    if i > 0:
        line = vtk.vtkLine()
        line.GetPointIds().SetId(0, i - 1)
        line.GetPointIds().SetId(1, i)
        lines.InsertNextCell(line)

polyData = vtk.vtkPolyData()
polyData.SetPoints(points)
polyData.SetLines(lines)

# Write the streamline to a VTK PolyData file (*.vtp)
writer = vtk.vtkXMLPolyDataWriter()
writer.SetFileName("streamline.vtp")
writer.SetInputData(polyData)
writer.Write()

Enter the x-coordinate of the seed point: 0
Enter the y-coordinate of the seed point: 0
Enter the z-coordinate of the seed point: 7


1

#### vtpFile has been generated named as streamline.vtp 

In [3]:
import plotly.graph_objects as go

streamline_points = np.array(streamline)  # Convert streamline points for plotting

fig = go.Figure()

# Add trace for the streamline
fig.add_trace(go.Scatter3d(
    x=streamline_points[:, 0],
    y=streamline_points[:, 1],
    z=streamline_points[:, 2],
    mode='lines',
    line=dict(color='green', width=2),
    name='Streamline'
))

# Set layout properties
fig.update_layout(
    title='3D Streamline Visualization',
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    )
)

# Show the interactive plot
fig.show()